# Detección de objetos usando Amazon rekognition

### Iniciamos las librerias necesarias

In [ ]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
bucketName = "{REPLACE_YOUR_BUCKET}"
imageName = "media/cars.png"

In [ ]:
s3.upload_file(
    Filename="resources/cars.png",
    Bucket=bucketName,
    Key=imageName,
)

In [ ]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

In [ ]:
detectLabelsResponse = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

In [ ]:
display(detectLabelsResponse)

In [ ]:
flaggedObjects = ["Car"]

for label in detectLabelsResponse["Labels"]:
    if(label["Name"] in flaggedObjects):
        print("Detected unsafe object:")
        print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
        print("  - Parents: {}".format(label["Parents"]))

In [ ]:
def drawBoundingBoxes (sourceImage, boxes, outputImage):
    colors = ((255,255,255),(255,255,255),(76,182,252),(52,194,123))
    imageLocation = 'resources/'+os.path.basename(sourceImage)
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
        imageFormat = 'JPEG'

    bbImage.save(outputImage,format=imageFormat)

    display(bbImage)

In [ ]:
boxes = []
objects = detectLabelsResponse['Labels']
for obj in objects:
    for einstance in obj["Instances"]:
        boxes.append ((obj['Name'], einstance['BoundingBox']))
    
drawBoundingBoxes(imageName, boxes, 'resources/cars2.png')

# Deteccion de objetos en video

In [ ]:
bucketName = "temporal-bucket-max"
videoName = "media/GrandTour720.mp4"

In [ ]:
s3.upload_file(
    Filename="resources/GrandTour720.mp4",
    Bucket=bucketName,
    Key=videoName,
)

In [ ]:
s3VideoUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': videoName})

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

In [ ]:
startLabelDetection = rekognition.start_label_detection(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

labelsJobId = startLabelDetection['JobId']
display("Job Id: {0}".format(labelsJobId))

In [ ]:
getObjectDetection = rekognition.get_label_detection(
    JobId=labelsJobId,
    SortBy='TIMESTAMP'
)

while(getObjectDetection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getObjectDetection = rekognition.get_label_detection(
    JobId=labelsJobId,
    SortBy='TIMESTAMP')
    
display(getObjectDetection['JobStatus'])

In [ ]:
flaggedObjectsInVideo = ["Car"]

theObjects = {}

strDetail = "Objects detected in video<br>=======================================<br>"
strOverall = "Objects in the overall video:<br>=======================================<br>"

for obj in getObjectDetection['Labels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['Label']["Confidence"]
    oname = obj['Label']["Name"]
    
    if(oname in flaggedObjectsInVideo):
        print("Found flagged object at {} ms: {} (Confidence: {})".format(ts, oname, round(cconfidence,2)))
    
    strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in theObjects:
        cojb = theObjects[oname]
        theObjects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        theObjects[oname] = {"Name" : oname, "Count": 1}

for theObject in theObjects:
    strOverall = strOverall + "Name: {}, Count: {}<br>".format(theObject, theObjects[theObject]["Count"])

display(HTML(strOverall))